<a href="https://colab.research.google.com/github/Utkarshmishra2k2/Focus_on_Vision-A_Statistical_Study_of_Eye/blob/main/Focus_on_Vision_A_Statistical_Study_of_Eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Standard Libraries
import numpy as np
import pandas as pd
import joblib
from collections import Counter

# Data Preprocessing and Transformation
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Statistical Analysis and Feature Selection
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning and Deep Learning
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow import keras

# Model Evaluation
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc

In [ ]:
data_01 = pd.read_excel('https://raw.githubusercontent.com/Utkarshmishra2k2/Focus_on_Vision-A_Statistical_Study_of_Eye/refs/heads/main/DATA:Focus%20on%20Vision_%20A%20Statistical%20Study%20of%20Eye.xlsx')

# Data Processing

In [ ]:
columns = {
    "1": "age",
    "2": "gender",
    "3": "occupation",
    "4": "education",
    "5": "environment",
    "6": "screen_hours",
    "7": "device",
    "8": "screen_use_purpose",
    "9": "screen_symptoms",
    "10": "reading_medium",
    "11": "reading_hours",
    "12": "night_mode",
    "13": "theme_usage",
    "14": "dark_usage",
    "15": "blue_light_usage",
    "16": "outdoor_activity",
    "17": "sunlight_hours",
    "18": "exercise_frequency",
    "19": "wear_sunglasses",
    "20": "lighting_conditions",
    "21": "take_breaks",
    "22": "sleep_hours",
    "23": "device_before_bedtime",
    "24": "eye_strain_reduction",
    "25": "nutrient_intake",
    "26": "air_quality",
    "27": "smoking_status",
    "28": "vision_correction",
    "29": "age_vision_correction",
    "30": "left_eye_power",
    "31": "right_eye_power",
    "32": "reason_vision_correction",
    "33": "screen_fatigue",
    "34": "activity_duration",
    "35": "eye_checkup_frequency",
    "36": "eye_self_care",
    "37": "future_vision_correction",
    "38": "corrective_procedures",
    "39": "surgical_procedure",
    "40": "surgery_age",
    "41": "procedure_effectiveness",
    "42": "parents_vision_problems",
    "43": "parent_with_vision_issues",
    "44": "parent_tools",
    "45": "parent_vision_problems",
    "46": "father_age_vision_correction",
    "47": "mother_age_vision_correction",
    "48": "relatives_vision_problems",
    "49": "relative_with_vision_issues",
    "50": "relative_tools",
    "51": "relative_vision_problems",
    "52": "relative_age_vision_correction",
    "53": "offspring_vision_problems",
    "54": "offspring_vision_problems_type",
    "55": "offspring_diagnosis_age",
    "56": "offspring_using_correction_tools",
    "57": "offspring_screen_time",
    "58": "offspring_outdoor_time",
    "59": "vision_influence_on_children",
    "60": "steps_to_prevent_children_vision",
    "61": "district"
}

In [ ]:
data_01.columns = data_01.columns.str.strip()
data_01.rename(columns=columns, inplace=True)

In [ ]:
data_01.dtypes

In [ ]:
data_01.isnull().sum()

In [ ]:
(data_01.isnull().sum() / len(data_01)) * 100

In [ ]:
data_01.describe(include='all').T

In [ ]:
data_01.info()

In [ ]:
def has_glasses(row):
    vc = row.get("vision_correction")
    print(f"Processing row: vision_correction = {vc}, left_eye_power = {row.get('left_eye_power')}, right_eye_power = {row.get('right_eye_power')}, age_vision_correction = {row.get('age_vision_correction')}")
    left_power = row.get("left_eye_power")
    right_power = row.get("right_eye_power")
    if (pd.isnull(left_power) or left_power == 0) and (pd.isnull(right_power) or right_power == 0):
        return "No"
    if pd.notnull(left_power) and left_power != 0:
        return "Yes"
    if pd.notnull(right_power) and right_power != 0:
        return "Yes"
    if pd.notnull(row.get("age_vision_correction")) and vc in ["Glasses", "Contact Lenses","Surgical Correction",'Vision Therapy']:
        return "Yes"
    return "No"
data_01["has_or_had_glasses"] = data_01.apply(has_glasses, axis=1)
print(data_01[["vision_correction", "left_eye_power", "right_eye_power", "age_vision_correction", "has_or_had_glasses"]].head())
print(data_01["has_or_had_glasses"].value_counts())

In [ ]:
print(data_01['has_or_had_glasses'].value_counts())

In [ ]:
all_missing_cols = data_01.columns[data_01.isnull().all()]

In [ ]:
df_dropped = data_01.drop(all_missing_cols, axis=1)

In [ ]:
data_01.drop(['air_quality'], axis=1,inplace=True)

In [ ]:
numerical_cols = df_dropped.select_dtypes(include=['number']).columns
categorical_cols = df_dropped.select_dtypes(exclude=['number']).columns

In [ ]:
numerical_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [ ]:
df_imputed = df_dropped.copy()
df_imputed[numerical_cols] = numerical_imputer.fit_transform(df_dropped[numerical_cols])
df_imputed[categorical_cols] = categorical_imputer.fit_transform(df_dropped[categorical_cols])

In [ ]:
data_01 = df_imputed.copy()

In [ ]:
categorical_columns = data_01.select_dtypes(include=['category', 'object']).columns
numerical_columns = data_01.select_dtypes(include=['number']).columns

In [ ]:
data_01[categorical_columns] = data_01[categorical_columns].apply(lambda x: x.astype('category'))

In [ ]:
for col in numerical_columns:
    data_01[col] = pd.to_numeric(data_01[col], errors='coerce')

In [ ]:
aqi_dict = {
    'Dombivali': 59,
    'Ghatkopar': 102,
    'Kurla':102,
    'Bhandup': 106,
    'Mumbai Suburban': 102,
    'Thane': 79,
    'Palghar': 89,
    'Mumbai': 85,
    'Vasai': 72,
    'Virar': 71,
    'Kalyan': 93
}

In [ ]:
data_01['air_quality'] = data_01['district'].astype(str).str.strip().map(aqi_dict)
data_01['air_quality'] = pd.to_numeric(data_01['air_quality'], errors='coerce')

# Statistical Analysis

In [ ]:
def normality_tests(data_01, numeric_columns):
    """
    Perform multiple normality tests on the numeric columns of a dataset.

    This function applies three statistical tests for normality to each numeric column in the
    input DataFrame `data_01`. It calculates the following tests:
    1. Shapiro-Wilk Test
    2. Kolmogorov-Smirnov (KS) Test
    3. Anderson-Darling Test

    For each test, the function computes the test statistic and p-value (where applicable).
    The results of these tests are stored in a list of dictionaries and then returned as
    a pandas DataFrame.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must have numeric columns that you want to test for normality.

    numeric_columns : list
        A list of column names (strings) in the DataFrame `data_01` that are numeric and should be tested for normality.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the normality tests for each numeric column.
        The DataFrame includes the following columns:
        - 'Column': Name of the column.
        - 'Shapiro-Wilk Statistic': The test statistic for the Shapiro-Wilk test.
        - 'Shapiro-Wilk p-value': The p-value for the Shapiro-Wilk test.
        - 'KS Statistic': The test statistic for the Kolmogorov-Smirnov test.
        - 'KS p-value': The p-value for the Kolmogorov-Smirnov test.
        - 'Anderson-Darling Statistic': The test statistic for the Anderson-Darling test.
        - 'Anderson-Darling Critical Values': The critical values from the Anderson-Darling test.
        - 'Anderson-Darling Significance Levels': The significance levels for the Anderson-Darling test.

    Example:
    --------
    normality_df = normality_tests(data_01, ['col1', 'col2', 'col3'])

    This will apply the three normality tests to the columns 'col1', 'col2', and 'col3'
    of the DataFrame `data_01` and return the results in `normality_df`.
    """

    normality_results = []
    for col in numeric_columns:
        shapiro_stat, shapiro_p = stats.shapiro(data_01[col].dropna())
        ks_stat, ks_p = stats.kstest(data_01[col].dropna(), 'norm', args=(data_01[col].mean(), data_01[col].std()))
        anderson_result = stats.anderson(data_01[col].dropna(), dist='norm')
        normality_results.append({
            'Column': col,
            'Shapiro-Wilk Statistic': shapiro_stat,
            'Shapiro-Wilk p-value': shapiro_p,
            'KS Statistic': ks_stat,
            'KS p-value': ks_p,
            'Anderson-Darling Statistic': anderson_result.statistic,
            'Anderson-Darling Critical Values': anderson_result.critical_values,
            'Anderson-Darling Significance Levels': anderson_result.significance_level
        })

    return pd.DataFrame(normality_results)
normality_tests(data_01, numerical_columns)

In [ ]:
fig = make_subplots(rows=7, cols=5, subplot_titles=numerical_columns) # Changed rows and cols
for i, col in enumerate(numerical_columns):
    data = data_01[col].dropna()
    (theoretical_q, sample_q), _ = stats.probplot(data, dist="norm")
    row = i // 5 + 1
    col_index = i % 5 + 1
    fig.add_trace(
        go.Scatter(
            x=theoretical_q,
            y=sample_q,
            mode='markers',
            name=col,
            showlegend=False
        ),
        row=row, col=col_index
    )
    min_val = min(theoretical_q.min(), sample_q.min())
    max_val = max(theoretical_q.max(), sample_q.max())
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False
        ),
        row=row, col=col_index
    )

fig.update_layout(height=2500, width=1200, title_text="Q-Q Plots")
fig.show()

In [ ]:
# normality = normality_tests(data_01, numerical_columns)
# normality.to_csv('normality.csv', index=False)

In [ ]:
def spearman_correlation(data_01, numeric_columns):
    """
    Calculate Spearman's rank correlation for each pair of numeric columns in the dataset.

    This function calculates the Spearman's rank correlation coefficient for each pair
    of numeric columns in the input DataFrame `data_01`. It computes both the correlation coefficient
    and the associated p-value for each pair.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must have numeric columns for correlation calculation.

    numeric_columns : list
        A list of column names (strings) in the DataFrame `data_01` that are numeric and should be used for
        Spearman's rank correlation.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the Spearman correlation tests for each pair of numeric columns.
        The DataFrame includes the following columns:
        - 'Column 1': Name of the first column in the pair.
        - 'Column 2': Name of the second column in the pair.
        - 'Spearman Correlation Coefficient': The Spearman correlation coefficient between the two columns.
        - 'Spearman p-value': The p-value associated with the Spearman correlation coefficient.

    Example:
    --------
    correlation_df = spearman_correlation(data_01, ['col1', 'col2', 'col3'])

    This will compute the Spearman's rank correlation for all pairs of columns 'col1', 'col2', and 'col3'
    in the DataFrame `data_01` and return the results in `correlation_df`.
    """

    correlation_results = []

    for i, col1 in enumerate(numeric_columns):
        for col2 in numeric_columns[i+1:]:
            spearman_corr, spearman_p = stats.spearmanr(data_01[col1].dropna(), data_01[col2].dropna())

            correlation_results.append({
                'Column 1': col1,
                'Column 2': col2,
                'Spearman Correlation Coefficient': spearman_corr,
                'Spearman p-value': spearman_p
            })

    return pd.DataFrame(correlation_results)
spearman_correlation(data_01, numerical_columns)

In [ ]:
def plot_spearman_correlation_matrix_seaborn(data_01, numeric_columns):
    """
    Plot the Spearman correlation matrix as a heatmap using Seaborn and Matplotlib.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must have numeric columns for correlation calculation.

    numeric_columns : list
        A list of column names (strings) in the DataFrame `data_01` that are numeric and should be used for
        Spearman's rank correlation.
    """
    spearman_corr_matrix = data_01[numeric_columns].corr(method='spearman')

    plt.figure(figsize=(10, 8))
    sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, cbar=True, facecolor='white')
    plt.title('Spearman Correlation Matrix')
    plt.gca().set_facecolor('grey')  # Set the axes background color to white
    plt.gcf().set_facecolor('grey')  # Set the figure background color to white
    plt.show()

plot_spearman_correlation_matrix_seaborn(data_01, numerical_columns)

In [ ]:
# correlation = spearman_correlation(data_01, numerical_columns)
# correlation.to_csv('correlation.csv', index=False)

In [ ]:
# Crosstab Analysis with Chi-Square Test
def chi_square_crosstab(data_01, categorical_columns):
    """
    Perform Chi-Square Test of Independence between each pair of categorical variables in the dataset.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must include categorical columns to be tested.

    categorical_columns : list of str
        A list of column names (strings) representing categorical columns to be tested.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the Chi-Square Test of Independence for each pair of categorical variables.
        The DataFrame includes the following columns:
        - 'Categorical Variable 1': The first categorical variable in the pair.
        - 'Categorical Variable 2': The second categorical variable in the pair.
        - 'Chi-Square Statistic': The Chi-Square test statistic.
        - 'p-value': The p-value for the Chi-Square test.
        - 'Degrees of Freedom': The degrees of freedom for the Chi-Square test.
    """

    chi_square_results = []

    for i, var1 in enumerate(categorical_columns):
        for var2 in categorical_columns[i + 1:]:
            contingency_table = pd.crosstab(data_01[var1], data_01[var2])

            chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

            chi_square_results.append({
                'Categorical Variable 1': var1,
                'Categorical Variable 2': var2,
                'Chi-Square Statistic': chi2_stat,
                'p-value': p_value,
                'Degrees of Freedom': dof
            })

    return pd.DataFrame(chi_square_results)
chi_square_crosstab(data_01, categorical_columns)

In [ ]:
# crosstab = chi_square_crosstab(data_01, categorical_columns)
# crosstab.to_csv('crosstab.csv', index=False)

In [ ]:
def chi_square_independence(data_01, categorical_columns):
    """
    Perform Chi-Square Test for Independence between pairs of categorical columns.

    This function calculates the Chi-Square Test for Independence for each pair of categorical columns in the
    input DataFrame `data_01`. It computes the Chi-Square statistic, p-value, degrees of freedom,
    and the expected frequency table for each pair.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must have categorical columns for the Chi-Square test.

    categorical_columns : list
        A list of column names (strings) in the DataFrame `data_01` that are categorical and should be used for
        the Chi-Square Test for Independence.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the Chi-Square Test for Independence for each pair of categorical columns.
        The DataFrame includes the following columns:
        - 'Column 1': Name of the first column in the pair.
        - 'Column 2': Name of the second column in the pair.
        - 'Chi-Square Statistic': The Chi-Square statistic for the test.
        - 'p-value': The p-value associated with the Chi-Square test.
        - 'Degrees of Freedom': The degrees of freedom for the test.
        - 'Expected Frequencies': The expected frequencies under the null hypothesis of independence.

    Example:
    --------
    chi_square_df = chi_square_independence(data_01, ['col1', 'col2', 'col3'])

    This will compute the Chi-Square Test for Independence for all pairs of columns 'col1', 'col2', and 'col3'
    in the DataFrame `data_01` and return the results in `chi_square_df`.
    """

    chi_square_results = []

    for i, col1 in enumerate(categorical_columns):
        for col2 in categorical_columns[i+1:]:
            contingency_table = pd.crosstab(data_01[col1], data_01[col2])

            chi2_stat, p_val, dof, expected = stats.chi2_contingency(contingency_table)

            chi_square_results.append({
                'Column 1': col1,
                'Column 2': col2,
                'Chi-Square Statistic': chi2_stat,
                'p-value': p_val,
                'Degrees of Freedom': dof,
                'Expected Frequencies': expected
            })

    return pd.DataFrame(chi_square_results)
chi_square_independence(data_01, categorical_columns)

In [ ]:
# independence = chi_square_independence(data_01, categorical_columns)
# independence.to_csv('independence.csv', index=False)

In [ ]:
def mann_whitney_u_test_all_pairs(data_01, numeric_columns, group_column):
    """
    Perform Mann-Whitney U Test between each pair of numeric variables, grouped by a categorical column.

    This function calculates the Mann-Whitney U Test between all pairs of numeric variables in `numeric_columns`,
    for each pair of groups defined by the `group_column`. It compares the distribution of each pair of numeric variables
    between the two groups and returns the results.

    Parameters:
    ----------
    data_01 : pandas.DataFrame
        The DataFrame containing the data. It must include numeric columns and a categorical column for grouping.

    numeric_columns : list of str
        A list of column names (strings) representing numeric columns to be tested.

    group_column : str
        The name of the categorical column used to group the data into two distinct groups for the Mann-Whitney U test.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the Mann-Whitney U Test for each pair of numeric variables.
        The DataFrame includes the following columns:
        - 'Numeric Variable 1': The first numeric variable in the pair.
        - 'Numeric Variable 2': The second numeric variable in the pair.
        - 'U-statistic': The Mann-Whitney U statistic for the test.
        - 'p-value': The p-value for the test.

    Example:
    --------
    mann_whitney_df = mann_whitney_u_test_all_pairs(data_01, ['col1', 'col2', 'col3'], 'group_col')

    This will compute the Mann-Whitney U Test for all pairs of numeric columns ['col1', 'col2', 'col3']
    in `data_01`, grouped by the `group_col`, and return the results in `mann_whitney_df`.
    """

    groups = data_01[group_column[0]].dropna().unique()

    mann_whitney_results = []

    for i, var1 in enumerate(numeric_columns):
        for var2 in numeric_columns[i+1:]:

            group1_data_var1 = data_01[data_01[group_column[0]] == groups[0]][var1].dropna()
            group2_data_var1 = data_01[data_01[group_column[0]] == groups[1]][var1].dropna()
            group1_data_var2 = data_01[data_01[group_column[0]] == groups[0]][var2].dropna()
            group2_data_var2 = data_01[data_01[group_column[0]] == groups[1]][var2].dropna()

            u_stat_var1, p_val_var1 = stats.mannwhitneyu(group1_data_var1, group2_data_var1, alternative='two-sided')
            u_stat_var2, p_val_var2 = stats.mannwhitneyu(group1_data_var2, group2_data_var2, alternative='two-sided')

            mann_whitney_results.append({
                'Numeric Variable 1': var1,
                'Numeric Variable 2': var2,
                'U-statistic (Var1)': u_stat_var1,
                'p-value (Var1)': p_val_var1,
                'U-statistic (Var2)': u_stat_var2,
                'p-value (Var2)': p_val_var2
            })

    return pd.DataFrame(mann_whitney_results)

mann_whitney_u_test_all_pairs(data_01, numerical_columns, categorical_columns)

In [ ]:
# mann_whitney = mann_whitney_u_test_all_pairs(data_01, numerical_columns, categorical_columns)
# mann_whitney.to_csv('mann_whitney.csv', index=False)

# Train - Test Split

In [ ]:
label_encoder = LabelEncoder()
for col in categorical_cols:  # Selects only object columns
    data_01[col] = label_encoder.fit_transform(data_01[col])

In [ ]:
for col in numerical_cols: #Select number columns
    data_01[col] = pd.to_numeric(data_01[col], errors='coerce')

In [ ]:
scaler = MinMaxScaler()
data_01[numerical_cols] = scaler.fit_transform(data_01[numerical_cols])

In [ ]:
X = data_01.drop(['has_or_had_glasses'], axis=1)
y = data_01['has_or_had_glasses']

In [ ]:
missing_values = X.isnull().sum()
print("Missing values before imputation:\n", missing_values[missing_values>0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=75)

In [ ]:
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
data_train.shape
data_train.info()

In [ ]:
data_test.shape
data_test.info()

In [ ]:
data_train.to_csv('data_train.csv', index=False)
data_test.to_csv('data_test.csv', index=False)

# Smote

In [ ]:
data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')

In [ ]:
class_distribution = data_train['has_or_had_glasses'].value_counts()
plt.figure(figsize=(8, 6))
class_distribution.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Class Distribution of has_or_had_glasses')
plt.xlabel('Has or Had Glasses')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

In [ ]:
class_percentages = (class_distribution / len(data_train)) * 100
print("Class Distribution:")
print(class_distribution)
print("\nClass Percentages:")
print(class_percentages)
if class_percentages.min() < 30:
    print("\nConclusion: The dataset is imbalanced.")
else:
    print("\nConclusion: The dataset is balanced or nearly balanced.")

In [ ]:
X = data_train.drop(['has_or_had_glasses'], axis=1)
y = data_train['has_or_had_glasses']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
smote = SMOTE(random_state=42, k_neighbors=5)

In [ ]:
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled_distribution = y_train_resampled.value_counts()
y_train_resampled_percentages = (y_train_resampled_distribution / len(y_train_resampled)) * 100

In [ ]:
num_synthetic_rows = X_train_resampled.shape[0] - X_train.shape[0]

In [ ]:
plt.subplot(1, 2, 2)
y_train_resampled_distribution.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Resampled Training Set Class Distribution')
plt.xlabel('Has or Had Glasses')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X_train.columns)
X_train_resampled_df['has_or_had_glasses'] = y_train_resampled

In [ ]:
data_01 = X_train_resampled_df.copy()

In [ ]:
data_01.to_csv('data_train_main.csv', index=False)

# Stepwise Logistic Regression

In [ ]:
data_train = pd.read_csv('data_train_main.csv')
data_test = pd.read_csv('data_test.csv')

In [ ]:
X_train = data_train.drop(['has_or_had_glasses'], axis=1)
y_train = data_train['has_or_had_glasses']

In [ ]:
X_test = data_test.drop(['has_or_had_glasses'], axis=1)
y_test = data_test['has_or_had_glasses']

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
print("X_train shape:", X_test.shape)
print("y_train shape:", y_test.shape)

In [ ]:
X_train_copy = X_train.copy()
y_train_copy = y_train.copy()

In [ ]:
sfs = SFS(LogisticRegression(solver='liblinear', penalty='l1'),  # Add L1 regularization
          k_features=10,
          forward=True,
          floating=True,
          scoring='accuracy',
          cv=5,
          n_jobs=-1)

In [ ]:
sfs = sfs.fit(X_train_copy, y_train_copy)

In [ ]:
selected_features = list(sfs.k_feature_names_)
print("Selected features:", selected_features)

# Logistic Regression

In [ ]:
X_train_selected = X_train_copy[selected_features]

In [ ]:
X_train_selected_with_constant = sm.add_constant(X_train_selected)

In [ ]:
logit_model = sm.Logit(y_train_copy, X_train_selected_with_constant).fit_regularized(method='l1')

In [ ]:
print(logit_model.summary())

In [ ]:
import plotly.express as px

ranked_features = {
    'reading_hours': 17.709124,
    'dark_usage': 2.341641,
    'age': 1.497100,
    'sleep_hours': 1.258598,
    'screen_hours': 0.659592,
    'occupation': 0.521482,
    'sunlight_hours': 0.448405,
    'lighting_conditions': 0.399558,
    'outdoor_activity': 0.201879,
    'exercise_frequency': 0.144893,
    'education': 0.070323
}

data = {'variable': list(ranked_features.keys()), 'importance': list(ranked_features.values())}

fig = px.pie(data, names='variable', values='importance',
             title='Updated Variable Contribution to Glasses Prediction',
             hover_data=['importance'],
             labels={'importance': 'Relative Importance'},
             color_discrete_sequence=px.colors.qualitative.Pastel,
             width=800,
             height=800)

fig.update_traces(textinfo='percent+label', pull=[0.03]*len(ranked_features))
fig.update_layout(title_font_size=20, title_x=0.5, uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

Assumption

In [ ]:
print("\n1. Binary Outcome:")
print("The target variable has values:", y_train.unique())
if len(y_train.unique()) == 2:
    print("Assumption met: The outcome is binary (0 and 1).")
else:
    print("Assumption NOT met: The outcome is not binary.")

In [ ]:
print("\n2. Independence of Observations:")

In [ ]:
print("\n3. Multicollinearity")
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_selected_with_constant.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_selected_with_constant.values, i)
                   for i in range(X_train_selected_with_constant.shape[1])]
print(vif_data)

In [ ]:
rank = np.linalg.matrix_rank(X_train_selected_with_constant)
print(f"Matrix rank: {rank}, Columns: {X_train_selected_with_constant.shape[1]}")

In [ ]:
coefficients = logit_model.params
p_values = logit_model.pvalues

In [ ]:
ranked_features = abs(coefficients).sort_values(ascending=False)
print("\nRanked Features by Absolute Coefficient Value:")
print(ranked_features)

# Model

In [ ]:
data_train = pd.read_csv('data_train_main.csv')
data_test = pd.read_csv('data_test.csv')

In [ ]:
X_train = data_train.drop(['has_or_had_glasses'], axis=1)
y_train = data_train['has_or_had_glasses']

In [ ]:
X_test = data_test.drop(['has_or_had_glasses'], axis=1)
y_test = data_test['has_or_had_glasses']

In [ ]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

In [ ]:
model.summary()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = model.predict(X_test)

In [ ]:
y_pred = (y_pred_probs > 0.5).astype(int)

In [ ]:
logloss = log_loss(y_test, y_pred_probs)

In [ ]:
weights = model.get_weights()

In [ ]:
first_layer_weights = weights[0]

In [ ]:
nn_coefficients = np.abs(first_layer_weights).sum(axis=1)

In [ ]:
feature_names = X_train.columns
for name, coef in zip(feature_names, nn_coefficients):
    print(f"{name}: {coef:.4f}")

In [ ]:
print(f'Neural Network Test Accuracy: {test_accuracy:.4f}')

In [ ]:
print(f'Log Loss (Calibration Check): {logloss:.4f}')

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
print("Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Print the AUC score
print(f"AUC Score: {roc_auc:.4f}")

In [ ]:
plt.figure(figsize=(10,6))
plt.barh(feature_names, nn_coefficients)
plt.xlabel("Importance (|weight sum|)")
plt.title("Feature Importance (NN)")
plt.tight_layout()
plt.show()

In [ ]:
print("The End")